In [1]:
#Importar librerias

import numpy as np
import itertools
from itertools import product
import pandas as pd

In [2]:
#Definir funciones a utilizar

#Función que devuelve las frecuencias de resonancia de los modos
def frecuencia_modos(nx, ny, nz, lx, ly, lz, c0=343):
    return (c0 / 2) * np.sqrt((nx / lx) ** 2 + (ny / ly) ** 2 + (nz / lz) ** 2)

#Función que clasifica los modos
def clasificar_modo(nx, ny, nz):
    num_ceros = sum([nx == 0, ny == 0, nz == 0])
    if num_ceros == 2:
        return "Axial"
    elif num_ceros == 1:
        return "Tangencial"
    else:
        return "Oblicuo"

#Función que identifica los modos degenerados
def modos_degenerados(frecuencias, tolerancia=1.0):
    degenerados = []
    n = len(frecuencias)
    usados = set()
    for i in range(n):
        for j in range(i+1, n):
            if abs(frecuencias[i][3] - frecuencias[j][3]) <= tolerancia:
                for k in range(j+1, n):
                    if abs(frecuencias[i][3] - frecuencias[j][3]) <= tolerancia and abs(frecuencias[j][3] - frecuencias[k][3]) <= tolerancia:
                        degenerados.append((frecuencias[i], frecuencias[j], frecuencias[k]))
                        usados.update([i, j, k])
                        break
                else:
                    if i not in usados and j not in usados:
                        degenerados.append((frecuencias[i], frecuencias[j]))
                        usados.update([i, j])
    return degenerados

In [3]:
# Dimensiones del recinto en metros
lx, ly, lz = 4.11, 5.86, 2.49

In [4]:
#Calcula los modos hasta una frecuencia deseada
frecuencias = []
N_max = 30
f_max = 300  # Frecuencia máxima deseada (Hz)

for nx in range(N_max + 1):
    for ny in range(N_max + 1):
        for nz in range(N_max + 1):
            if (nx, ny, nz) == (0, 0, 0):
                continue  # Ignora el modo trivial
            f_m = frecuencia_modos(nx, ny, nz, lx, ly, lz)
            if f_m <= f_max:
                frecuencias.append((nx, ny, nz, f_m))

# ordenar por frecuencia
frecuencias.sort(key=lambda x: x[3])

In [5]:
# Imprime las 30 frecuencias más bajas

print("Las 30 frecuencias más bajas son:")
for i, (nx, ny, nz, f) in enumerate(frecuencias[:30], 1):
    tipo = clasificar_modo(nx, ny, nz)
    print(f"{i}. Modo ({nx},{ny},{nz}): {f:.2f} Hz - {tipo}")

Las 30 frecuencias más bajas son:
1. Modo (0,1,0): 29.27 Hz - Axial
2. Modo (1,0,0): 41.73 Hz - Axial
3. Modo (1,1,0): 50.97 Hz - Tangencial
4. Modo (0,2,0): 58.53 Hz - Axial
5. Modo (0,0,1): 68.88 Hz - Axial
6. Modo (1,2,0): 71.88 Hz - Tangencial
7. Modo (0,1,1): 74.84 Hz - Tangencial
8. Modo (1,0,1): 80.53 Hz - Tangencial
9. Modo (2,0,0): 83.45 Hz - Axial
10. Modo (1,1,1): 85.68 Hz - Oblicuo
11. Modo (0,3,0): 87.80 Hz - Axial
12. Modo (2,1,0): 88.44 Hz - Tangencial
13. Modo (0,2,1): 90.39 Hz - Tangencial
14. Modo (1,3,0): 97.21 Hz - Tangencial
15. Modo (1,2,1): 99.55 Hz - Oblicuo
16. Modo (2,2,0): 101.94 Hz - Tangencial
17. Modo (2,0,1): 108.21 Hz - Tangencial
18. Modo (0,3,1): 111.59 Hz - Tangencial
19. Modo (2,1,1): 112.09 Hz - Oblicuo
20. Modo (0,4,0): 117.06 Hz - Axial
21. Modo (1,3,1): 119.14 Hz - Oblicuo
22. Modo (2,3,0): 121.13 Hz - Tangencial
23. Modo (2,2,1): 123.02 Hz - Oblicuo
24. Modo (1,4,0): 124.28 Hz - Tangencial
25. Modo (3,0,0): 125.18 Hz - Axial
26. Modo (3,1,0): 12

In [6]:
# Identificar modos degenerados entre las primeras 30 frecuencias
tolerancia = 1.0 # Tolerancia en Hz
degenerados = modos_degenerados(frecuencias[:30], tolerancia)
print("\nModos degenerados dentro de la tolerancia de 1.0 Hz:")
for modo in degenerados:
    if len(modo) == 2:
        (nx1, ny1, nz1, f1), (nx2, ny2, nz2, f2) = modo
        print(f"Modo ({nx1},{ny1},{nz1}) - {f1:.2f} Hz y Modo ({nx2},{ny2},{nz2}) - {f2:.2f} Hz")
    elif len(modo) == 3:
        (nx1, ny1, nz1, f1), (nx2, ny2, nz2, f2), (nx3, ny3, nz3, f3) = modo
        print(f"Modo ({nx1},{ny1},{nz1}) - {f1:.2f} Hz , Modo ({nx2},{ny2},{nz2}) - {f2:.2f} Hz y Modo ({nx3},{ny3},{nz3}) - {f3:.2f} Hz")


Modos degenerados dentro de la tolerancia de 1.0 Hz:
Modo (0,3,0) - 87.80 Hz y Modo (2,1,0) - 88.44 Hz
Modo (0,3,1) - 111.59 Hz y Modo (2,1,1) - 112.09 Hz
Modo (1,4,0) - 124.28 Hz y Modo (3,0,0) - 125.18 Hz
Modo (0,0,2) - 137.75 Hz y Modo (3,2,0) - 138.19 Hz


In [7]:
# Clasificar y guardar en listas
nx_list, ny_list, nz_list, f_list, tipo_list = [], [], [], [], []

for nx, ny, nz, f in frecuencias:
    tipo = clasificar_modo(nx, ny, nz)
    nx_list.append(nx)
    ny_list.append(ny)
    nz_list.append(nz)
    f_list.append(f)
    tipo_list.append(tipo.lower())  # convertir a minúsculas para compatibilidad

# Crear DataFrame
modos_df = pd.DataFrame({
    'nx': nx_list,
    'ny': ny_list,
    'nz': nz_list,
    'frecuencia': f_list,
    'tipo': tipo_list
})

# Guardar como CSV
modos_df.to_csv("modos_lab.csv", index=False)
print("Archivo 'modos_lab.csv' exportado correctamente.")

Archivo 'modos_lab.csv' exportado correctamente.
